In [3]:
# 설치하는 데 2~3분 정도의 시간이 소요됩니다. 설치를 완료하고 세션을 다시 시작하세요.
!pip install -q nltk langchain langchain-community langchain-core unstructured sentence_transformers faiss-gpu openai selenium pypdf pacmap plotly_express nbformat rank_bm25 kiwipiepy bitsandbytes accelerate

^C
ERROR: Operation cancelled by user

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [1]:
!pip install gdown


  Using cached gdown-5.2.0-py3-none-any.whl.metadata (5.8 kB)
Using cached gdown-5.2.0-py3-none-any.whl (18 kB)

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [52]:
import requests
from bs4 import BeautifulSoup
import time
import re

def crawl_disu_notice():
    """
    - https://www.disu.ac.kr/community/notice?cidx=44&page=1
    - 목록 페이지의 <caption>게시물 목록</caption> 아래 <tr class="noti">를 순회하여 게시글 정보를 수집
    """
    base_url = "https://www.disu.ac.kr"
    list_url = "https://www.disu.ac.kr/community/notice"
    
    all_posts = []
    page = 1
    
    while True:
        params = {
            'cidx': 44,
            'page': page
        }
        response = requests.get(list_url, params=params)
        if not response.ok:
            print(f"페이지 {page} 요청 실패. 상태코드: {response.status_code}")
            break
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 1) <caption>게시물 목록</caption>을 포함하는 table 찾기
        table = None
        for t in soup.find_all('table'):
            caption_tag = t.find('caption')
            if caption_tag and '게시물 목록' in caption_tag.get_text():
                table = t
                break
        
        if not table:
            print(f"목록 테이블을 찾지 못했습니다. (page={page})")
            break
        
        tbody = table.find('tbody')
        if not tbody:
            print(f"테이블에 <tbody>가 없습니다. (page={page})")
            break
        
        # 2) 공지사항 목록 <tr class="noti">
        rows = tbody.find_all('tr', class_='noti')
        if not rows:
            print(f"더 이상 'tr.noti' 게시글이 없습니다. (page={page})")
            break
        
        for row in rows:
            # <td class="title noti-tit"> 안에 <a href="..."> 링크
            title_td = row.find('td', class_='title noti-tit')
            if not title_td:
                continue
            
            link_tag = title_td.find('a')
            if not link_tag:
                continue
            
            title = link_tag.get_text(strip=True)
            href = link_tag.get('href')  # "?md=v&bbsidx=7997" 형태
            
            if not href:
                continue
            
            # 상세 페이지 URL 구성
            # 실제 링크 구조에 따라 조합이 달라질 수 있음
            # 예) "/community/notice?md=v&bbsidx=7997"인 경우:
            if href.startswith('http'):
                post_url = href
            elif href.startswith('/'):
                post_url = base_url + href
            else:
                # 파라미터만 붙는 형태("?md=v&bbsidx=7997")라면:
                post_url = base_url + "/community/notice" + href
            
            # 날짜나 기타 정보가 필요하면 추가적으로 다른 <td>에서 추출
            # 예: 작성일이 <td class="date">2024.01.01</td> 형태라면
            date_td = row.find('td', class_='date')
            date_str = date_td.get_text(strip=True) if date_td else ""
            
            # 상세 페이지 본문 크롤링
            content_text = crawl_post_detail(post_url)
            
            post_data = {
                'title': title,
                'date': date_str,
                'url': post_url,
                'content': content_text
            }
            all_posts.append(post_data)
        
        print(f"{page} 페이지 처리 완료. 현재까지 수집된 게시글 수: {len(all_posts)}")
        
        page += 1
        # 서버 부하 방지를 위해 잠시 대기
        time.sleep(1)
    
    return all_posts

def crawl_post_detail(url):
    """
    상세 페이지에서 본문을 추출하는 예시 함수.
    실제 HTML 구조(예: <div class="bbs_view"> 등)를 확인해 적절한 선택자를 사용해야 함.
    """
    response = requests.get(url)
    if not response.ok:
        print(f"상세 페이지 요청 실패: {url}")
        return ""
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # <div class="bbs_view"> 또는 <div class="view_contents"> 등 본문 영역 파악
    content_div = soup.find('div', class_='bbs_view')
    if not content_div:
        content_div = soup.find('div', class_='view_contents')
    if not content_div:
        return ""
    
    # 텍스트만 추출 + 불필요한 공백/줄바꿈 정리
    text = content_div.get_text(separator="\n", strip=True)
    text = re.sub(r'\s+', ' ', text)
    
    return text

if __name__ == "__main__":
    posts = crawl_disu_notice()
    print(f"\n총 크롤링된 게시글 수: {len(posts)}")
    for p in posts[:3]:
        print("="*80)
        print(f"제목: {p['title']}")
        print(f"날짜: {p['date']}")
        print(f"URL : {p['url']}")
        print(f"본문(일부) : {p['content'][:100]}...")


1 페이지 처리 완료. 현재까지 수집된 게시글 수: 10
더 이상 'tr.noti' 게시글이 없습니다. (page=2)

총 크롤링된 게시글 수: 10
제목: 차세대반도체 페스티벌 POLARIS SIF 2025 개최 안내
날짜: 
URL : https://www.disu.ac.kr/community/notice?md=v&bbsidx=7997
본문(일부) : ...
제목: '시스템반도체입문' K-MOOC 수강신청 기간 및 개강 안내
날짜: 
URL : https://www.disu.ac.kr/community/notice?md=v&bbsidx=7976
본문(일부) : ...
제목: [중앙대학교] 2024학년도 동계 계절학기 차세대반도체학과 학점교류 개설교과목 및 학점교류 안내
날짜: 
URL : https://www.disu.ac.kr/community/notice?md=v&bbsidx=7955
본문(일부) : ...


In [53]:
posts

[{'title': '차세대반도체 페스티벌 POLARIS SIF 2025 개최 안내',
  'date': '',
  'url': 'https://www.disu.ac.kr/community/notice?md=v&bbsidx=7997',
  'content': ''},
 {'title': "'시스템반도체입문' K-MOOC 수강신청 기간 및 개강 안내",
  'date': '',
  'url': 'https://www.disu.ac.kr/community/notice?md=v&bbsidx=7976',
  'content': ''},
 {'title': '[중앙대학교] 2024학년도 동계 계절학기 차세대반도체학과 학점교류 개설교과목 및 학점교류 안내',
  'date': '',
  'url': 'https://www.disu.ac.kr/community/notice?md=v&bbsidx=7955',
  'content': ''},
 {'title': "'SoC구조및설계' K-MOOC 수강신청 기간 및 개강 안내",
  'date': '',
  'url': 'https://www.disu.ac.kr/community/notice?md=v&bbsidx=7907',
  'content': ''},
 {'title': '[상시신청] 마이크로디그리 인증서 발급 절차 방식 변경 안내 (7.29~ 현재 신청가능)',
  'date': '',
  'url': 'https://www.disu.ac.kr/community/notice?md=v&bbsidx=7755',
  'content': ''},
 {'title': '[FAQ] 중앙대학교 차세대반도체학과 자주 묻는 질문 [24.08.08 업데이트]',
  'date': '',
  'url': 'https://www.disu.ac.kr/community/notice?md=v&bbsidx=7515',
  'content': ''},
 {'title': '[FAQ] 학점교류 취소 신청 방법 안내(상시신청 X)',
  'date': ''

차세대반도체학과 사이트 html 사이트, 반도체이야기 사이트 등.. 한국어 온라인.

In [14]:
import gdown
#https://drive.google.com/file/d/11A_vNUXVt5gq4KSJjn-Lq00c1FxyUVAY/view?usp=sharing
url = '11A_vNUXVt5gq4KSJjn-Lq00c1FxyUVAY'
output = 'AI.pdf'
gdown.download(id=url, output=output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=11A_vNUXVt5gq4KSJjn-Lq00c1FxyUVAY
To: /home/elicer/SIF2025/AI.pdf
100%|██████████| 17.6M/17.6M [00:01<00:00, 14.7MB/s]


'AI.pdf'

In [31]:
%pip install -qU langchain_community beautifulsoup4

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langchain_community.document_loaders import UnstructuredURLLoader

import nltk

nltk.download("punkt")
nltk.download("averaged_perceptron_tagger")


urls = [
    "https://www.disu.ac.kr/academic/program",
    "https://news.samsungsemiconductor.com/kr/behind-the-chip-%eb%b0%98%eb%8f%84%ec%b2%b4-%ea%b7%b8-%ec%84%b1%ec%9e%a5%ec%9d%98-%ea%b8%b0%eb%a1%9d-%ea%b3%bc%ea%b1%b0-%ed%98%84%ec%9e%ac-%ea%b7%b8%eb%a6%ac%ea%b3%a0-%eb%af%b8%eb%9e%98/"

]
loader = UnstructuredURLLoader(urls=urls)
docs = loader.load()

[nltk_data] Downloading package punkt to /home/kyudan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/kyudan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [38]:
docs

[Document(metadata={'source': 'https://www.disu.ac.kr/academic/program'}, page_content='혁신융합대학\n\n혁신융합대학이란?\n\n인사말\n\n7대목표\n\n인재상\n\nFAQ\n\n참여대학/조직도\n\n오시는 길\n\n학위학사 안내\n\n학위제도\n\n개설교과목\n\n학사일정\n\n성과확산센터\n\n소개\n\nPOLAR explorer\n\nPOLAR expert\n\nPOLAR W-square\n\nPOLAR edu\n\n경진대회\n\nPOLARIS LOC\n\nPOLARIS LOS\n\n경진대회\n\nTCAT\n\nSIF 2025\n\n소개\n\n지난 SIF 보기\n\n게시판\n\n공지사항\n\nNews\n\n행사\n\nQ&A\n\nPOLARIS TMI\n\nPOLAR GATE\n\nPOLARIS 학사학위안내\n\n학위제도\n\n학위학사 안내\n\n학위제도\n\n실무 인재\n\n현장중심형 교육을 통한 실무형 인재\n\n초급 전문대생 및 일반인 반도체 입문(마이크로디그리)(9학점)\n\n융합 인재\n\n본 소속 전공 외 반도체 분야의 지식을 겸비한 융합형 이공계 비전공 인재\n\n초급 중급 고급 이공계, 비이공계 학생 및 일반인 이공계 및 신기술분야 초급과정 이수 비이공계 학생 * 이에 준하는 수준의 학생 참여 가능 신기술분야 초급과정을 이수한 학생 * 이에 준하는 수준의 학생 참여 가능 반도체 초급 (마이크로디그리)(9학점) 반도체 중급(분야별 마이크로디그리)(9학점) 연계전공(42학점 이상) 부전공(21학점 이상) 융합전공(42학점 이상)\n\n통합 인재\n\n반도체와 연관된 업무를 원활히 수행 할 수 있는 인문/사회계 인재\n\n초급 비이공계 학생 및 일반인 반도체 입문(마이크로디그리)(9학점)\n\n전문 인재\n\n산업체 현장의 문제를 스스로 해결하는 능력과 창의력을 보유한 리더형의 반도체 전문가\n\n고급 전문 신기술분야 초급과정을 이수한 학생 * 이에 준하는 수준의 학생 참여 가

In [39]:
from langchain_community.document_loaders import PyPDFium2Loader
file_path = "차세대반도체.pdf"
loader = PyPDFium2Loader(file_path)
pages = loader.load_and_split()

pages

/home/kyudan/.local/lib/python3.10/site-packages/pypdfium2/_helpers/textpage.py:80: UserWarning:

get_text_range() call with default params will be implicitly redirected to get_text_bounded()



[Document(metadata={'source': '차세대반도체.pdf', 'page': 0}, page_content='♧ Ctrl+F로 검색할 수 있습니다. ♧\r\n1 중앙대학교에서 개설되지 않은 과목도 수강가능한가요?\r\n→ 첨단분야 혁신융합대학은 7개 대학(중앙대 외 서울대, 숭실대, 포항공대, 강원대,\r\n대구대, 조선이공대)에서 교과목을 신청, 이수 가능하고 학점교류 형식으로 신청할\r\n수 있습니다.\r\n2 강원대나 포스텍같이 중앙대에서 먼 곳은 직접 가서 수업을 들어야 되나요?\r\n→ 지리적으로 먼 곳은 정규학기에는 대부분 줌(실시간)으로 수업을 합니다.\r\n서울대나 숭실대도 대부분 줌으로 수업을 하지만, 계절학기의 경우는 오프라인\r\n수업일 때도 있습니다. 학교별, 교수님별로 진행 방식이 다르니, 각 학기의 학교별\r\n학점교류 신청 공지사항 및 강의계획서를 참고해주시기 바랍니다. 강의에 대한\r\n문의는 담당대학 사무실에 직접 문의바랍니다.\r\n- 차세대반도체 컨소시엄 조직도 안내 (https://url.kr/ju6ndr)\r\n3 반도체와 연관이 없는 전공이라 화학, 물리, 소자 자식이 없는데 해당 과목 수강에 지장이\r\n없나요?\r\n→ 차세대반도체학과에서는 비전공자 및 비이공계 학생들 또한 수강할 수 있도록\r\n난이도별로 교과목이 구성되어 있습니다. 교양과목 및 초급난이도의 과목도\r\n개설됩니다.\r\n4 마이크로디그리와 다전공 학위가 무엇인가요?\r\n● 차세대반도체 융합대학 과정을 통해서 총 16개의 마이크로디그리(이수증)와 부전공,\r\n복수전공 학위를 취득할 수 있습니다.(연계전공 추가 진행 예정)\r\n→ 마이크로디그리란 기존 학위(4년 또는 2년) 제도와 달리 이수시간이나 학점\r\n단위로 이수 결과를 조합해서 학위를 취득하는 제도입니다. 차세대반도체학과에서'),
 Document(metadata={'source': '차세대반도체.pdf', 'page': 1}, page_content='개설된 과목으로 

In [40]:
docs = docs+pages

In [41]:
docs

[Document(metadata={'source': 'https://www.disu.ac.kr/academic/program'}, page_content='혁신융합대학\n\n혁신융합대학이란?\n\n인사말\n\n7대목표\n\n인재상\n\nFAQ\n\n참여대학/조직도\n\n오시는 길\n\n학위학사 안내\n\n학위제도\n\n개설교과목\n\n학사일정\n\n성과확산센터\n\n소개\n\nPOLAR explorer\n\nPOLAR expert\n\nPOLAR W-square\n\nPOLAR edu\n\n경진대회\n\nPOLARIS LOC\n\nPOLARIS LOS\n\n경진대회\n\nTCAT\n\nSIF 2025\n\n소개\n\n지난 SIF 보기\n\n게시판\n\n공지사항\n\nNews\n\n행사\n\nQ&A\n\nPOLARIS TMI\n\nPOLAR GATE\n\nPOLARIS 학사학위안내\n\n학위제도\n\n학위학사 안내\n\n학위제도\n\n실무 인재\n\n현장중심형 교육을 통한 실무형 인재\n\n초급 전문대생 및 일반인 반도체 입문(마이크로디그리)(9학점)\n\n융합 인재\n\n본 소속 전공 외 반도체 분야의 지식을 겸비한 융합형 이공계 비전공 인재\n\n초급 중급 고급 이공계, 비이공계 학생 및 일반인 이공계 및 신기술분야 초급과정 이수 비이공계 학생 * 이에 준하는 수준의 학생 참여 가능 신기술분야 초급과정을 이수한 학생 * 이에 준하는 수준의 학생 참여 가능 반도체 초급 (마이크로디그리)(9학점) 반도체 중급(분야별 마이크로디그리)(9학점) 연계전공(42학점 이상) 부전공(21학점 이상) 융합전공(42학점 이상)\n\n통합 인재\n\n반도체와 연관된 업무를 원활히 수행 할 수 있는 인문/사회계 인재\n\n초급 비이공계 학생 및 일반인 반도체 입문(마이크로디그리)(9학점)\n\n전문 인재\n\n산업체 현장의 문제를 스스로 해결하는 능력과 창의력을 보유한 리더형의 반도체 전문가\n\n고급 전문 신기술분야 초급과정을 이수한 학생 * 이에 준하는 수준의 학생 참여 가

In [6]:
contents = [doc.page_content for doc in docs]
all_toc = [c.split("→")[-1].split("\n\n")[1:] for c in contents]
all_toc[0]

['삼성반도체이야기는 더 이상 Internet Explorer를 지원하지 않습니다. 최적의 환경을 위해 다른 웹브라우저 사용을 권장합니다.',
 'ENG',
 '기술 >',
 '반도체+',
 '[Behind the CHIP] 반도체, 그 성장의 기록: 과거, 현재, 그리고 미래',
 '[Behind the CHIP] 반도체, 그 성장의 기록: 과거, 현재, 그리고 미래',
 '2024.11.20',
 'by.세상의모든지식',
 '페이스북',
 '트위터',
 '카카오톡',
 '이메일',
 '반도체는 전기가 통하는 도체와 전기가 통하지 않는 부도체, 두 가지의 성질을 임의로 조절할 수 있는 물질로, 전자 신호를 제어하고 처리하는 데 필수적인 역할을 한다. 이러한 특성 덕분에 스마트폰부터 컴퓨터, 자동차 부품까지 반도체는 현대 전자 산업의 핵심이 되며, 오늘날 우리가 사용하는 대부분의 전자 장치의 근간을 이루고 있다.',
 '반도체 기술 수준이 국가 경쟁력의 척도가 될 만큼 그 중요성이 급부상하고 있는 요즘, 반도체 기술이 어디서부터 시작해 현재 첨단 반도체 기술로까지 발전할 수 있었는지 반도체의 역사와 현주소, 그리고 앞으로의 전망에 대해 알아보도록 하자.',
 '역사의 시작, 최초의 반도체 발견',
 '1833년, 영국의 저명한 과학자 ‘마이클 패러데이’는 비금속 물질의 전기 전도성을 연구하다가 황화은(Ag₂S)의 독특한 전기적 특성을 발견했다. 당시 패러데이는 ‘전기와 자기의 관계’에 대한 실험을 진행하고 있었으며, 특히 전기 전도에 대한 실험을 통해 물질의 전기 전도성을 조사하고 있었다.',
 '그는 금속과 비금속 재료를 포함한 다양한 물질의 전도성을 비교하며, 전도성 고체에 전류를 흐르게 하는 실험 과정에서 금속은 온도가 올라가면 저항이 증가하는 반면, 황화은과 같은 비금속 재료는 온도가 올라가도 전류가 지속적으로 흐른다는 점을 발견했다.',
 '해당 연구는 추후 반도체의 개념으로 발전하게 되는 중요한 발견이 되었으며, 패러데이는 이를 통해 금속과 비금속이

In [44]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 이미 docs = docs + pages 로 문서가 합쳐져 있다는 가정 하에
# 1. TextSplitter 설정: chunk_size, chunk_overlap, 길이 판단 함수 등
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,      # 청크 하나의 최대 문자 수
    chunk_overlap=40,    # 청크 간 겹치는 문자 수 (필요시 조정)
    length_function=len,  # 길이 측정 함수
)

# 2. 청크 생성
chunked_docs = text_splitter.split_documents(docs)

In [45]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy

embedding_model_name = "snunlp/KR-SBERT-V40K-klueNLI-augSTS"
embedding_model = HuggingFaceEmbeddings(
    model_name=embedding_model_name,
    encode_kwargs={"normalize_embeddings": True},
)
vectorstore = FAISS.from_documents(
    documents=chunked_docs, embedding=embedding_model, distance_strategy=DistanceStrategy.EUCLIDEAN_DISTANCE
)

vector_count = vectorstore.index.ntotal
print(f"저장된 벡터의 개수: {vector_count}")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


저장된 벡터의 개수: 115


In [46]:
user_query = "진공관 다이오드는 누가 발명했지?"
query_vector = embedding_model.embed_query(user_query)

In [47]:
print(f"임베딩 차원 수: {len(query_vector)}")

임베딩 차원 수: 768


In [12]:
import pacmap
import numpy as np
import plotly.express as px

embedding_projector = pacmap.PaCMAP(n_components=2, n_neighbors=None, MN_ratio=0.5, FP_ratio=2.0, random_state=133)

embeddings_2d = [list(vectorstore.index.reconstruct_n(idx, 1)[0]) for idx in range(len(chunked_docs))] + [query_vector]

# Fit the data (the index of transformed data corresponds to the index of the original data)
documents_projected = embedding_projector.fit_transform(np.array(embeddings_2d), init="pca")

/home/kyudan/anaconda3/envs/RAG/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)


In [ ]:
import pandas as pd

df = pd.DataFrame.from_dict(
    [
        {
            "x": documents_projected[i, 0],
            "y": documents_projected[i, 1],
            "source": chunked_docs[i].metadata["source"].split("ko/")[-1],
            "extract": chunked_docs[i].page_content[:100] + "...",
            "symbol": "circle",
            "size_col": 4,
        }
        for i in range(len(chunked_docs))
    ]
    + [
        {
            "x": documents_projected[-1, 0],
            "y": documents_projected[-1, 1],
            "source": "User query",
            "extract": user_query,
            "size_col": 2,
            "symbol": "star",
        }
    ]
)

# Visualize the embedding
fig = px.scatter(
    df,
    x="x",
    y="y",
    color="source",
    hover_data="extract",
    size="size_col",
    symbol="symbol",
    color_discrete_map={"User query": "black"},
    width=1100,
    height=800,
)
fig.update_traces(
    marker=dict(opacity=1, line=dict(width=0, color="DarkSlateGrey")),
    selector=dict(mode="markers"),
)
fig.update_layout(
    legend_title_text="<b>Chunk source</b>",
    title="<b>2D Projection of Chunk Embeddings via PaCMAP</b>",
)
fig.show()

In [14]:
user_query = "진공관 다이오드는 누가 발명했지?"

In [15]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
basic_docs = retriever.invoke(user_query)
basic_docs

[Document(metadata={'source': 'https://news.samsungsemiconductor.com/kr/behind-the-chip-%eb%b0%98%eb%8f%84%ec%b2%b4-%ea%b7%b8-%ec%84%b1%ec%9e%a5%ec%9d%98-%ea%b8%b0%eb%a1%9d-%ea%b3%bc%ea%b1%b0-%ed%98%84%ec%9e%ac-%ea%b7%b8%eb%a6%ac%ea%b3%a0-%eb%af%b8%eb%9e%98/'}, page_content='집적회로(IC)의 탄생과 무어의 법칙: 한동안 트랜지스터가 진공관의 수많은 문제를 해결하긴 했지만, 또 다른 문제는 여전히 남아 있었다. 전자 회로는 트랜지스터, 저항, 다이오드, 캐패시터 등 여러 전자부품을 수작업으로 기판에 납땜해 연결하는 방식으로 만들어졌는데, 이러한 방식은 생산 시간이 매우 오래 걸렸다.'),
 Document(metadata={'source': 'https://news.samsungsemiconductor.com/kr/behind-the-chip-%eb%b0%98%eb%8f%84%ec%b2%b4-%ea%b7%b8-%ec%84%b1%ec%9e%a5%ec%9d%98-%ea%b8%b0%eb%a1%9d-%ea%b3%bc%ea%b1%b0-%ed%98%84%ec%9e%ac-%ea%b7%b8%eb%a6%ac%ea%b3%a0-%eb%af%b8%eb%9e%98/'}, page_content='한동안 트랜지스터가 진공관의 수많은 문제를 해결하긴 했지만, 또 다른 문제는 여전히 남아 있었다. 전자 회로는 트랜지스터, 저항, 다이오드, 캐패시터 등 여러 전자부품을 수작업으로 기판에 납땜해 연결하는 방식으로 만들어졌는데, 이러한 방식은 생산 시간이 매우 오래 걸렸다.: 본문 바로가기\n\n삼성반도체이야기는 더 이상 Internet Explorer를 지원하지 않습니다. 최적의 환경을 위해 다른 웹브라우저 사용을 권장합니다.\n\nENG\n\n기술 >\n\n반도체+\n\n[Be

In [23]:
basic_docs[0].page_content

'집적회로(IC)의 탄생과 무어의 법칙: 한동안 트랜지스터가 진공관의 수많은 문제를 해결하긴 했지만, 또 다른 문제는 여전히 남아 있었다. 전자 회로는 트랜지스터, 저항, 다이오드, 캐패시터 등 여러 전자부품을 수작업으로 기판에 납땜해 연결하는 방식으로 만들어졌는데, 이러한 방식은 생산 시간이 매우 오래 걸렸다.'

In [24]:
YOUR_OPENAI_API_KEY = 

In [48]:
# 1. 사용자가 물어본 질의
user_query = "차세대반도체학과가 뭐지?"

# 2. VectorStore에서 top-k개 문서 검색
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
basic_docs = retriever.invoke(user_query)

# 3. 문서 내용들을 합쳐서 context로 구성
context = "\n".join([doc.page_content for doc in basic_docs])  

# 4. 실제로 OpenAI ChatCompletion API를 호출하여 답변 생성
from openai import OpenAI

client = OpenAI(
    api_key=YOUR_OPENAI_API_KEY
)

# question과 context를 message에 삽입
completion = client.chat.completions.create(
    model="gpt-4o-mini",  
    messages=[
        {
            "role": "user",
            "content": f"""당신은 반도체와 차세대반도체학과에 대해서 설명하는 Assistant입니다.
차세대반도체학과는 한국 서울의 중앙대학교의 창의ICT공과대학에 설립된 학과입니다. 
다음 맥락에 맞게 질문에 답하세요.

맥락: {context}

질문: {user_query}
"""
        }
    ]
)

# 5. 결과 출력
print(completion.choices[0].message.content)

차세대반도체학과는 한국 서울의 중앙대학교 창의ICT공과대학에 설립된 학과로, 반도체 기술의 발전과 새로운 혁신을 다루는 전문 교육 프로그램을 제공합니다. 이 학과는 전자공학 발전에서 중요한 역할을 하는 반도체 기술에 집중하며, 차세대 반도체 소자와 공정, 응용 분야에 관한 깊이 있는 이론과 실습을 학습합니다. 학생들은 최신 기술 트렌드와 함께 실무 경험을 쌓아가며, 반도체 산업에서 경쟁력을 갖출 수 있는 인재로 성장할 수 있는 기회를 제공합니다.
